<div style="width:90%; text-align:center; border-width: 0px; display:block; margin-left:auto; margin-right:auto;">
<div class="alert alert-block alert-success" style="text-align:center; color:navy;">
<img src="https://raw.githubusercontent.com/bgeneto/MCA/main/imagens/logo_unb.png" style="width: 200px; opacity:0.85;">
<h1>Universidade de Brasília</h1>
<h2>Instituto de Física</h2>
<hr style="width:44%;border:1px solid navy;">
<h3>Métodos Computacionais A (MCA)</h3> 
<h4>Prof. Bernhard Enders</h4>
<hr style="width:44%;border:1px solid navy;">
</div>
</div>

# **➲ Aula 03 - Erros Numéricos**

## ➥ Limitações da representação em ponto flutuante
---

Erros de arredondamento e truncamento são comuns em análise numérica (usando Python ou qualquer outra linguagem de programação), especialmente quando números em ponto flutuante são usados. 

Tal situação existe devido à limitação da representação binária em ponto flutuante, que estabelece uma quantidade fixa de bits para o expoente, a mantissa e o sinal do número real a ser representado (vide norma* [IEEE 754](https://en.wikipedia.org/wiki/IEEE_754-1985)).

><b>*OBSERVAÇÕES:</b>
>A norma IEEE 754 é um padrão para representação e aritmética de números em ponto flutuante. Essa norma é amplamente utilizada em computação e define o formato de representação de números de ponto flutuante em sistemas de hardware e software.
>
>Os principais pontos da norma IEEE 754 são:
>
> - Existem dois formatos principais para números de ponto flutuante: simples precisão (32 bits) e dupla precisão (64 bits).
>
> - Os números são representados em notação científica normalizada com mantissa e expoente binários.
>
> - A precisão de um número de ponto flutuante é determinada pelo número de bits na mantissa, o que afeta o número de dígitos significativos que podem ser armazenados. **(7 dígitos em precisão simples, 14 dígitos com precisão dupla)**
>
> - A norma define valores especiais para representar números negativos, infinito, NaN (*not a number*) e zero.
>
> - A aritmética de ponto flutuante segue regras específicas para lidar com valores especiais, estouro de precisão, subfluxo e operações de conversão.
>
> - A norma também define uma série de funções matemáticas, como seno, cosseno, exponencial e logaritmo, que devem ser implementadas de acordo com padrões específicos.
>
>A norma IEEE 754 é importante para garantir que os números de ponto flutuante sejam representados e calculados de maneira **consistente** em diferentes sistemas de hardware e software. Isso é essencial para garantir a **interoperabilidade e a portabilidade** de aplicativos que envolvem cálculos numéricos.


Seguem abaixo alguns exemplos de situações em que esses erros podem ocorrer:

1. Comparação de números de ponto flutuante: Devido a erros de arredondamento, duas expressões que deveriam ser iguais podem não ser iguais em um cálculo numérico. Por exemplo:

In [239]:
x = 0.1 + 0.1 + 0.1
y = 0.3
print(x == y)  # retorna False

False


Vamos verificar que o mesmo tipo de problema ocorre também usando Fortran:

In [240]:
%%writefile erro.f90

program erro
    implicit none
    real :: x, y
    
    x = 0.1 + 0.1 + 0.1 + 0.1 + 0.1 + 0.1 + 0.1
    y = 0.7
    print*, x == y
    
end program erro

Overwriting erro.f90


In [241]:
%%bash
gfortran erro.f90 -o erro.x
./erro.x

 F


2. Truncamento de números grandes: Números grandes podem ser truncados quando são convertidos para ponto flutuante. Isso pode levar a erros em cálculos que envolvem números grandes. Por exemplo:

In [242]:
x = 10**20 + 1.0
y = x - 10**20
print(y)  # retorna 0.0 em vez de 1.0

0.0


O mesmo ocorre em Fortran:

In [243]:
%%writefile trunca.f90

program trunca
    implicit none
    real :: x, y
    
    x = 1e20 + 1.0
    y = x - 1e20
    print*, y
    
end program trunca

Overwriting trunca.f90


In [244]:
%%bash
gfortran trunca.f90 -o trunca.x
./trunca.x

   0.00000000    


Erros acumulados em operações repetidas: Erros de arredondamento podem se acumular em cálculos que envolvem operações repetidas, especialmente quando números de ponto flutuante não-exatamente representáveis são utilizados. Por exemplo:

In [245]:
x = 0.1
for i in range(100):
    x += 0.1

print(x)

10.09999999999998


Usando Fortran:

In [246]:
%%writefile repete.f90

program repete
    implicit none
    integer :: i
    double precision :: x
    
    x = 0.1
    do i = 1, 100
        x = x + 0.1
    end do

    print*, x
    
end program repete

Overwriting repete.f90


In [247]:
%%bash
gfortran repete.f90 -o repete.x
./repete.x

   10.100000150501728     


## ➥ Evitando erros numéricos com Python
---

Em geral, não devemos comparar números em ponto flutuante usando o operador de igualdade `==` , mas sim a função `isclose()` da biblioteca padrão **`math`**:

In [248]:
import math

x = 0.1 + 0.1 + 0.1
y = 0.3

math.isclose(x, y) 

True

Se você estiver trabalhando com matrizes numéricas em Python, a biblioteca NumPy inclui a função `isclose()` que pode ser usada para comparar elementos de matrizes em ponto flutuante com uma margem de tolerância especificada. Por exemplo, `numpy.isclose(x, y, rtol=1e-9, atol=0.0)` verifica se os elementos de x e y são aproximadamente iguais com uma precisão relativa de 1e-9 e uma tolerância absoluta de 0.0.

Em relação ao problema de truncamento, devemos procurar realizar as operações em uma ordem mais adequada para a máquina ou utilizar uma biblioteca de maior precisão ou precisão arbitrária:

In [249]:
from decimal import *
getcontext()

Context(prec=64, rounding=ROUND_HALF_EVEN, Emin=-999999, Emax=999999, capitals=1, clamp=0, flags=[FloatOperation], traps=[InvalidOperation, DivisionByZero, Overflow])

In [250]:
x = Decimal(10**20 + 1)
y = Decimal(x - 10**20)
print(y)

1


In [251]:
getcontext().prec = 64
x = Decimal(0.1)
for i in range(100):
    x += Decimal(0.1)

print(x)

10.1000000000000005606626274357040529139339923858642578125


Também podemos utilizar a biblioteca **`mpmath`**, mas antes precisamos instalar a biblioteca usando os comandos `pip` ou `conda`:

In [252]:
# instalando a biblioteca de múltipla precisão
import sys
!{sys.executable} -m pip install mpmath

In [253]:
# importando a biblioteca e mostrando as configurações padrão
from mpmath import *
print(mp)

Mpmath settings:
  mp.prec = 336               [default: 53]
  mp.dps = 100                [default: 15]
  mp.trap_complex = False     [default: False]


In [254]:
# aumentando o número de precisão das casas de decimais 
mp.dps = 100
mp.prec  # mostra o número de bits necessários

336

In [255]:
x = mpf(0.1)
for i in range(100):
    x += mpf(0.1)

print(x)

10.1000000000000005606626274357040529139339923858642578125


## ➥ Erros numéricos na prática
---

Vamos exemplificar a problemática trazida pelos erros numéricos em um problema prático. 

<div class="alert alert-block alert-info">
<b>&#9997; Exemplo:</b> Escreva um programa que calcule a área de um triângulo qualquer dados os seus lados.
</div>

Para tanto, vamos utilizar a [Fórmula de Herão](https://pt.wikipedia.org/wiki/Teorema_de_Her%C3%A3o) de Alexandria, a qual computa a área baseada apenas nos lados a, b e c do triângulo (e em seu semiperímetro):

![triangulo-abc](https://raw.githubusercontent.com/bgeneto/MCA/main/imagens/triangulo-abc.png)

$$
A = \sqrt{p(p-a)(p-b)(p-c)}
$$



In [256]:
def area_triangulo(a, b, c):
    '''
    Utiliza a fórmula de Herão para calcular a área de um triângulo
    dados os seus lados.
    ''' 
    s = 0.5*(a + b + c)
    return math.sqrt(s*(s-a)*(s-b)*(s-c))


def area_triangulo_retangulo(b, h):
    '''
    Utiliza a fórmula tradicional para calcular a área de um triângulo
    retângulo.
    ''' 
    return 0.5*b*h

In [257]:
# área do triângulo retângulo de lados: 3, 4 e 5
area_triangulo(3, 4, 5)

6.0

In [258]:
# vamos comparar com a fórmula tracional p/ um triângulo retângulo
area_triangulo_retangulo(4, 3)

6.0

Vamos calcular agora a área dos seguintes triângulos:

| a           | b           | c           |
| ----------- | ----------- | ----------- |
| 100000      | 99999.99979 | 0.00029     |
| 100000      | 100000      | 1.00005     |
| 99999.99996 | 99999.99994 | 0.00003     |
| 31622.77662 | 0.000023    | 31622.77661 |


In [259]:
areas = []
a = area_triangulo(100000, 99999.99979, 0.00029)
areas.append(a)

In [260]:
a = area_triangulo(100000, 100000, 1.00005)
areas.append(a)

In [261]:
a = area_triangulo(99999.99996, 99999.99994, 0.00003)
areas.append(a)

In [262]:
a = area_triangulo(31622.77662, 0.000023, 31622.77661)
areas.append(a)
areas

[9.999999809638329, 50002.49999949365, 1.1180333905130204, 0.3274905327782571]

É possível confiar nesses resultados? Será que esses resultados estão corretos? Se estão corretos, até quantas casas depois da vírgula? Existe algum método numericamente mais preciso e estável? Sim, existe, graças a W. Kahan:

In [263]:
    def area_triangulo_kahan(a, b, c):
        '''
        Utiliza a fórmula de W. Kahan (mais precisa que a fórmula de Herão)
        para calcular a área de um triângulo qualquer (dados os lados).
        '''
        # precisamos garantir a seguinte ordenação para os lados: a ≥ b ≥ c
        a, b, c = sorted([a, b, c], reverse=True)
        
        # atenção: não remova nenhum parênteses!
        return 0.25 * math.sqrt((a + (b + c)) * (c - (a - b)) * (c + (a - b)) * (a + (b - c)))


In [264]:
areas_kahan = []
a = area_triangulo_kahan(100000, 99999.99979, 0.00029)
areas_kahan.append(a)

In [265]:
a = area_triangulo_kahan(100000, 100000, 1.00005)
areas_kahan.append(a)

In [266]:
a = area_triangulo_kahan(99999.99996, 99999.99994, 0.00003)
areas_kahan.append(a)

In [267]:
a = area_triangulo_kahan(31622.77662, 0.000023, 31622.77661)
areas_kahan.append(a)
areas_kahan

[10.000000077021038,
 50002.499999374915,
 1.1180336853952004,
 0.3274904599426237]

In [268]:
# calcula o erro absoluto da fórmula de Heron
import numpy as np
np.array(areas) - np.array(areas_kahan)

array([-2.67382710e-07,  1.18736352e-07, -2.94882180e-07,  7.28356334e-08])